# Definitions

In [1]:
# imports
from typing import Tuple, List

In [2]:
# constants
DATASET_NAMES = ["evalplus/humanevalplus", "dz1/CodeScore-HumanEval-ET"]
MODEL_NAMES = ["Qwen/Qwen2.5-1.5B-Instruct", "lmsys/vicuna-7b-v1.5"]
NUM_PROMPTS = 10
SEED = 42
INDICES = [124,  31,  59,  76,  81,   7,  89,  25, 121, 112]

In [3]:
# set environmental variables to suppress warnings
import os

os.environ["HF_ALLOW_CODE_EVAL"] = "1"
os.environ["TOKENIZERS_PARALLELISM"]= "false"

In [4]:
import re

def extract_python_code(cot_response: str) -> list[str]:
    """
    Extracts all Python code blocks from a string.

    Args:
        cot_response: The string containing the LLM response.

    Returns:
        A list of strings, where each string is a Python code block.
    """
    # The `re.DOTALL` flag is crucial for matching across multiple lines.
    pattern = r"```python\n(.*?)```"
    matches = re.findall(pattern, cot_response, re.DOTALL)

    # Clean up the captured code by removing leading/trailing whitespace
    return [code.strip() for code in matches]

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from evaluate import load
import torch

def create_cot_prompt(prompt: str) -> str:
  cot_prompt = f"Implement the python function given by this definition:\n\n{prompt}\n\nLet's think step by step.\n"

  return cot_prompt

class PassKEvaluator:
  def __init__(
    self,
    model_name: str,
    dtype='auto',
    device_map='auto',
    quantization_config=None
  ):
    # instance variables
    self.model_name = model_name
    self.metric = load("code_eval")

    # create model
    self.model = AutoModelForCausalLM.from_pretrained(
      self.model_name,
      dtype=dtype,
      device_map=device_map,
      quantization_config=quantization_config
    )

    # set model to eval mode
    self.model.eval()

    # create tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

  def evaluate(
    self,
    test_cases: List[str],
    prompt: str,
    k: List[int]=[1, 10, 20]
  ):
    # get largest (usually last) k value
    largest_k = max(k)

    # query model wth prompts and output responses
    generated_prompts = self.generate(prompt, largest_k)

    # parse python code
    candidates = []

    for p in generated_prompts:
        extracted = extract_python_code(p)
        if len(extracted) > 0:
            candidates.append(extracted[0])
        else:
            candidates.append('')

    # compute pass@k metric
    results = self.metric.compute(
      references=test_cases,
      predictions=[candidates],
      k=k
    )

    return results, candidates

  def generate(self, prompt: str, num_samples: int) -> List[str]:
    # tokenize prompt/input
    inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

    # Generate multiple samples using a high temperature for diversity
    outputs = self.model.generate(
        **inputs,
        do_sample=True,
        temperature=0.8,
        num_return_sequences=num_samples,
        max_new_tokens=300
    ).to('cpu')

    # return code examples/responses as list of strings
    return [
        self.tokenizer.decode(
          out,
          skip_special_tokens=True
        )
      for out in outputs
    ]

  def __del__(self):
    del self.model
    del self.tokenizer
    del self.metric
    torch.cuda.empty_cache()

In [6]:
import multiprocessing as mp
from tqdm import trange
import json

# evaluation process
def run_evals(ev, ds, prefix):
    for i in trange(len(ds)):
        try:
            res = {}
    
            sample = ds[i]
    
            prompt = create_cot_prompt(sample['prompt'])
            test = sample['test']
            results, candidates = ev.evaluate(
                test,
                prompt,
                [1,5]
            )
    
            print(results)
    
            res['prompt'] = prompt
            res['test'] = test
            res['results'] = results
            res['candidates'] = candidates
    
            with open(f'{prefix}_{i}.json', "w") as f:
                json.dump(res, f, indent=4)
        except Exception as e:
            print(e)
        finally:
            continue

# Loading the Dataset

In [7]:
from datasets import load_dataset

In [8]:
# load humanevalplus dataset
humanevalplus_ds = load_dataset(DATASET_NAMES[0])['test']

# Finding Test Cases

In [9]:
# create 1.5B parameter qwen model
ev = PassKEvaluator(MODEL_NAMES[0])

In [ ]:
# run qwen on dataset

# file to log to
file_prefix = "cot_results"

run_evals(ev, humanevalplus_ds, file_prefix)

  1%|▎                                        | 1/164 [00:30<1:23:09, 30.61s/it]

list index out of range


  1%|▌                                        | 2/164 [01:00<1:21:44, 30.28s/it]

list index out of range


  2%|▊                                        | 3/164 [01:15<1:02:54, 23.44s/it]

In [9]:
# create vicuna model to evluate and use quantization so can run on 8gb VRAM hardware
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16  # Recommended for speed
)

ev = PassKEvaluator(MODEL_NAMES[1], quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# run vicuna on dataset
from tqdm import trange
import json

# file to log to
file_prefix = "cot_results_llama"

# run on llama model
run_evals(ev, humanevalplus_ds, file_prefix)

  1%|▎                                        | 1/164 [00:29<1:21:23, 29.96s/it]

list index out of range


  1%|▌                                        | 2/164 [00:59<1:19:37, 29.49s/it]

list index out of range


  2%|▊                                        | 3/164 [01:27<1:18:22, 29.21s/it]

...

{'pass@1': np.float64(1.0), 'pass@5': np.float64(1.0)}


 93%|████████████████████████████████████▍  | 153/164 [1:15:41<05:40, 30.93s/it]

list index out of range


 94%|████████████████████████████████████▌  | 154/164 [1:16:13<05:12, 31.27s/it]

list index out of range


# Question 2 Experiments

In [7]:
# find results where there were failures and success cases
from glob import glob
import json

# placehold for cases
cases = []

for p in glob("test_data/*.json"):
    with open(p, 'r') as f:
        # load json file
        payload = json.load(f)
        res = payload['results']

        # create relevant info dict
        info = {}
        info['prompt'] = payload['prompt']
        info['pass@1'] = res[0]['pass@1']
        info['pass@5'] = res[0]['pass@5']
        info['result'] = res[1]
        info['test'] = payload['test']

        # append model information
        if (f.name.split('_')[3] == 'llama'):
            info['model'] = 'vicuna'
        else:
            info['model'] = 'qwen'

        # append pass/fail information
        if (res[0]['pass@1'] < 1.0):
            info['status'] = 'fail'
        else:
            info['status'] = 'pass'

        # append to all cases
        cases.append(info)

In [8]:
import pandas as pd

# create results for creating tables
tabular_results = [{k:v for k,v in info.items() if k != 'result'} for info in cases]

# separate results for each model
tab_results_qwen = pd.DataFrame([res for res in tabular_results if res['model'] == 'qwen'])
tab_results_vicuna = pd.DataFrame([res for res in tabular_results if res['model'] == 'vicuna'])

# pick two failures and eight successes from for each model's results
tab_results_qwen = pd.concat([tab_results_qwen[tab_results_qwen['pass@5'] < 1][:2], tab_results_qwen[tab_results_qwen['status'] == 'pass'][:8]], axis=0)
tab_results_vicuna = pd.concat([tab_results_vicuna[tab_results_vicuna['pass@1'] < 1][:2], tab_results_vicuna[tab_results_vicuna['status'] == 'pass'][:8]], axis=0)

# reset indices
tab_results_qwen.reset_index(inplace=True, drop=True)
tab_results_vicuna.reset_index(inplace=True, drop=True)

tab_results_qwen.to_excel('qwen_cot.xlsx', index=False)
tab_results_vicuna.to_excel('vicuna_cot.xlsx', index=False)

In [9]:
tab_results_qwen

,prompt,pass@1,pass@5,test,model,status
0,Implement the python function given by this de...,0.4,0.97619,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,fail
1,Implement the python function given by this de...,0.1,0.50000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,fail
2,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
3,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
4,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
5,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
6,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
7,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
8,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass
9,Implement the python function given by this de...,1.0,1.00000,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,qwen,pass


In [10]:
tab_results_vicuna

,prompt,pass@1,pass@5,test,model,status
0,Implement the python function given by this de...,0.8,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,fail
1,Implement the python function given by this de...,0.8,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,fail
2,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
3,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
4,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
5,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
6,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
7,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass
8,Implement the python function given by this de...,1.0,1.0,import numpy as np\n\ndef is_floats(x) -> bool...,vicuna,pass
9,Implement the python function given by this de...,1.0,1.0,\n\nimport numpy as np\n\ndef is_floats(x) -> ...,vicuna,pass


In [11]:
failed_prompts_qwen = tab_results_qwen[tab_results_qwen['status'] == 'fail']['prompt']
qwen_failures = [(info['result'], info['prompt'], info['pass@1'], info['pass@5'], info['test'])
 for info in cases if info['prompt'] == failed_prompts_qwen[0] or
 info['prompt'] == failed_prompts_qwen[1]
][:2]

In [12]:
failed_prompts_vicuna = tab_results_vicuna[tab_results_vicuna['status'] == 'fail']['prompt']
vicuna_failures = [(info['result'], info['prompt'], info['pass@1'], info['pass@5'], info['test'])
 for info in cases if info['prompt'] == failed_prompts_vicuna[0] or
 info['prompt'] == failed_prompts_vicuna[1]
][:2]

In [13]:
for fail, prompt, k1, k5, _ in qwen_failures:
    for f in fail:
        print(prompt)
        print(k1)
        print(k5)
        for results in fail['0']:
            result = results[-1]
            if result['result'].startswith('failed'):
                print(result['result'])
    print()

Implement the python function given by this definition:

from typing import List


def sort_numbers(numbers: str) -> str:
    """ Input is a space-delimited string of numberals from 'zero' to 'nine'.
    Valid choices are 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight' and 'nine'.
    Return the string with numbers sorted from smallest to largest
    >>> sort_numbers('three one five')
    'one three five'
    """


Let's think step by step.

0.4
0.9761904761904762
failed: invalid literal for int() with base 10: 'three'
failed: invalid literal for int() with base 10: 'three'
failed: '1'
failed: invalid literal for int() with base 10: 'three'
failed: 
failed: list indices must be integers or slices, not list

Implement the python function given by this definition:

from typing import List


def parse_music(music_string: str) -> List[int]:
    """ Input to this function is a string representing musical notes in a special ASCII format.
    Your task is to parse this 

In [14]:
for fail, prompt, k1, k5, _ in vicuna_failures:
    for f in fail:
        print(prompt)
        print(k1)
        print(k5)
        for results in fail['0']:
            result = results[-1]
            if result['result'].startswith('failed'):
                print(result['result'])
    print()

Implement the python function given by this definition:

from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """


Let's think step by step.

0.5999999999999999
1.0
failed: name 'List' is not defined
failed: name 'List' is not defined
failed: name 'List' is not defined
failed: name 'List' is not defined

Implement the python function given by this definition:

from typing import List, Tuple


def rolling_max(numbers: List[int]) -> List[int]:
    """ From a given list of integers, generate a list of rolling maximum element found until given moment
    in the sequence.
    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])
    [1, 2, 3, 3, 3, 4, 4]
    """


Let's think step by step.

0.8
1.0
failed: name 'List' is not defined



In [17]:
# load qwen model
ev = PassKEvaluator(MODEL_NAMES[0])

In [34]:
# first qwen prompt
results, candidates = ev.evaluate(
    [qwen_failures[0][-1]],
    qwen_failures[0][1] + ' convert the input to a list of numbers before sorting them.',
    [1,5]
)

In [23]:
print('original: ', qwen_failures[0][1])
print()
print('new: ', qwen_failures[0][1] + ' convert the input to a list of numbers before sorting them.')

original:  Implement the python function given by this definition:

from typing import List


def sort_numbers(numbers: str) -> str:
    """ Input is a space-delimited string of numberals from 'zero' to 'nine'.
    Valid choices are 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight' and 'nine'.
    Return the string with numbers sorted from smallest to largest
    >>> sort_numbers('three one five')
    'one three five'
    """


Let's think step by step.


new:  Implement the python function given by this definition:

from typing import List


def sort_numbers(numbers: str) -> str:
    """ Input is a space-delimited string of numberals from 'zero' to 'nine'.
    Valid choices are 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight' and 'nine'.
    Return the string with numbers sorted from smallest to largest
    >>> sort_numbers('three one five')
    'one three five'
    """


Let's think step by step.
 convert the input to a list of numbers befor

In [39]:
print("new results: ", results[0]['pass@1'], results[0]['pass@5'])
print("old results: ", qwen_failures[0][2], qwen_failures[0][3])

new results:  1.0 1.0
old results:  0.4 0.9761904761904762


In [40]:
# second qwen prompt
results, candidates = ev.evaluate(
    [qwen_failures[1][-1]],
    qwen_failures[1][1] + ' Include necessary imports in the generated python code',
    [1,5]
)

In [24]:
print('original: ', qwen_failures[1][1])
print()
print('new: ', qwen_failures[1][1] + ' convert the input to a list of numbers before sorting them.')

original:  Implement the python function given by this definition:

from typing import List


def parse_music(music_string: str) -> List[int]:
    """ Input to this function is a string representing musical notes in a special ASCII format.
    Your task is to parse this string and return list of integers corresponding to how many beats does each
    not last.

    Here is a legend:
    'o' - whole note, lasts four beats
    'o|' - half note, lasts two beats
    '.|' - quater note, lasts one beat

    >>> parse_music('o o| .| o| o| .| .| .| .| o o')
    [4, 2, 1, 2, 2, 1, 1, 1, 1, 4, 4]
    """


Let's think step by step.


new:  Implement the python function given by this definition:

from typing import List


def parse_music(music_string: str) -> List[int]:
    """ Input to this function is a string representing musical notes in a special ASCII format.
    Your task is to parse this string and return list of integers corresponding to how many beats does each
    not last.

    Here is

In [42]:
print("new results: ", results[0]['pass@1'], results[0]['pass@5'])
print("old results: ", qwen_failures[1][2], qwen_failures[1][3])

new results:  0.8 1.0
old results:  0.09999999999999998 0.5


In [15]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16  # Recommended for speed
)

ev = PassKEvaluator(MODEL_NAMES[1], quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [21]:
# first vicuna prompt
results, candidates = ev.evaluate(
    [vicuna_failures[0][-1]],
    vicuna_failures[0][1] + ' Include necessary imports in the generated python code before the python function definition',
    [1,5]
)

In [25]:
print('original: ', vicuna_failures[0][1])
print()
print('new: ', vicuna_failures[0][1] + ' convert the input to a list of numbers before sorting them.')

original:  Implement the python function given by this definition:

from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """


Let's think step by step.


new:  Implement the python function given by this definition:

from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """


Let's think step by step.
 convert the input to a list of numbers before sorting them.


In [22]:
print("new results: ", results[0]['pass@1'], results[0]['pass@5'])
print("old results: ", vicuna_failures[0][2], vicuna_failures[0][3])

new results:  1.0 1.0
old results:  0.5999999999999999 1.0


In [19]:
# second vicuna prompt
results, candidates = ev.evaluate(
    [vicuna_failures[1][-1]],
    vicuna_failures[1][1] + ' Include necessary imports in the generated python code before the python function definition',
    [1,5]
)

In [26]:
print('original: ', vicuna_failures[1][1])
print()
print('new: ', vicuna_failures[1][1] + ' convert the input to a list of numbers before sorting them.')

original:  Implement the python function given by this definition:

from typing import List, Tuple


def rolling_max(numbers: List[int]) -> List[int]:
    """ From a given list of integers, generate a list of rolling maximum element found until given moment
    in the sequence.
    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])
    [1, 2, 3, 3, 3, 4, 4]
    """


Let's think step by step.


new:  Implement the python function given by this definition:

from typing import List, Tuple


def rolling_max(numbers: List[int]) -> List[int]:
    """ From a given list of integers, generate a list of rolling maximum element found until given moment
    in the sequence.
    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])
    [1, 2, 3, 3, 3, 4, 4]
    """


Let's think step by step.
 convert the input to a list of numbers before sorting them.


In [20]:
print("new results: ", results[0]['pass@1'], results[0]['pass@5'])
print("old results: ", vicuna_failures[1][2], vicuna_failures[1][3])

new results:  1.0 1.0
old results:  0.8 1.0
